# Aries Basic Controller - Bob

## This notebook works through the present proof protocol from the provers's perspective, it should be run alongside the [verifier](http://localhost:8888/notebooks/verifier.ipynb) notebook from Alices's perspective. 

## Before running through these two notebooks you should run through the Issuer/Holder flow found in these two notebooks - [issuer](http://localhost:8888/notebooks/issuer.ipynb) [holder](http://localhost:8889/notebooks/holder.ipynb).

If unfamiliar with the present-proof protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0037-present-proof)

## Complete steps 1-3 in the [verifier](http://localhost:8888/notebooks/verifier.ipynb) notebook first.

## 4. Instatiate the controller for Bob's Agent

In [1]:
%autoawait
import time
import asyncio

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

IPython autoawait is `on`, and set to use `asyncio`


## 5. Verify that Bob has at least one credential in his wallet

This uses the credential api, for more details see this [notebook](http://localhost:8889/notebooks/credential-api.ipynb). 
A credential with the specified credential_id is issued and saved in the issue-credential flow that can be run through  by following the [issuer](http://localhost:8888/notebooks/issuer.ipynb) and [holder](http://localhost:8889/notebooks/holder.ipynb) notebooks. (See step 12. store the credential, in the holder notebook for the credential id)

**This step will error out otherwise**

In [4]:
credential_id = "My OM Credential"
credential = await agent_controller.credentials.get_by_id(credential_id)
print(credential)



{'referent': 'My OM Credential', 'attrs': {'name': 'Bob', 'skill': 'researcher', 'age': '21'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'rev_reg_id': None, 'cred_rev_id': None}


## 6. Register Listeners

The handler should get called every time the controller receives a webhook with the topic present_proof, printing out the payload. The agent calls to this webhook every time it receives a present proof protocol message from another agent. 

The code for this protocol can be found [here](https://github.com/hyperledger/aries-cloudagent-python/tree/master/aries_cloudagent/protocols/issue_credential).

In [7]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    print("Handle present proof")
    print(payload)

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)

Handle present proof
{'state': 'request_received', 'connection_id': '35370789-764f-443a-9526-94044dbdc702', 'role': 'prover', 'updated_at': '2020-09-03 11:59:11.295947Z', 'initiator': 'external', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_name_uuid': {'name': 'name', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_skill_uuid': {'name': 'skill', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_country_uuid': {'name': 'country'}}, 'requested_predicates': {'0_age_GE_uuid': {'name': 'age', 'p_type': '>=', 'p_value': 21, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '1012453674509974792060177'}, 'thread_id': 'd0dec549-b119-4f05-9384-1edfcd634bc5', 'created_at': '2020-09-03 11:59:11.295947Z', 'presentation_exchange_id': 'a7336bbc-488c-4294-982c-2558726ccec9', 'trace': False}


## 7. Continue in the [verifier](http://localhost:8888/notebooks/verifier.ipynb) notebook.

This sends a proof request to Bob

## 8. Fetch the record of the proof request

### TODO refactor to use handler.

In [8]:
response = await agent_controller.proofs.get_records()
print(response)

print('\n')

state = response['results'][0]["state"]
presentation_exchange_id = response['results'][0]['presentation_exchange_id']
presentation_request = response['results'][0]['presentation_request']

print('Presentation Exchange ID\n')
print(response['results'][0]['presentation_exchange_id'])
print('Presentation Request Object\n')
print(response['results'][0]['presentation_request'])
print('Requested Attributes\n')
print(response['results'][0]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
print('Requested Predicates\n')
print(response['results'][0]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

{'results': [{'state': 'request_received', 'connection_id': '35370789-764f-443a-9526-94044dbdc702', 'role': 'prover', 'updated_at': '2020-09-03 11:59:11.295947Z', 'initiator': 'external', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_name_uuid': {'name': 'name', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_skill_uuid': {'name': 'skill', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_country_uuid': {'name': 'country'}}, 'requested_predicates': {'0_age_GE_uuid': {'name': 'age', 'p_type': '>=', 'p_value': 21, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '1012453674509974792060177'}, 'thread_id': 'd0dec549-b119-4f05-9384-1edfcd634bc5', 'created_at': '2020-09-03 11:59:11.295947Z', 'presentation_exchange_id': 'a7336bbc-488c-4294-982c-2558726ccec9', 'trace': False}]}


Presentation Exchange ID

a7336bbc-488c-4294-982c-2558726ccec9
Presentation Request Object

{'name'

## 9. Fetch relevant credentials for proof presentation

### TODO: Make clearer. Better documentation. Again this is way too complex. We need to think of ways to abstract away this complexity. For now breaking it doen and documenting each part in markdown may be helpful.

In [12]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "South Africa"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
proof = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(proof)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

Received Request -> Query for credentials in the wallet that satisfy the proof request
[{'cred_info': {'referent': 'My OM Credential', 'attrs': {'age': '21', 'name': 'Bob', 'skill': 'researcher'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'rev_reg_id': None, 'cred_rev_id': None}, 'interval': None, 'presentation_referents': ['0_name_uuid', '0_age_GE_uuid', '0_skill_uuid']}]

Generate the proof
{'requested_predicates': {'0_age_GE_uuid': {'cred_id': 'My OM Credential'}}, 'requested_attributes': {'0_name_uuid': {'cred_id': 'My OM Credential', 'revealed': True}, '0_skill_uuid': {'cred_id': 'My OM Credential', 'revealed': True}}, 'self_attested_attributes': {'0_country_uuid': 'South Africa'}}

XXX
{'0_age_GE_uuid': {'cred_id': 'My OM Credential'}}
{'0_name_uuid': {'cred_id': 'My OM Credential', 'revealed': True}, '0_skill_uuid': {'cred_id': 'My OM Credential', 'revealed': True}}
{'0_country_uuid': 'South Afr

## 10. Send Proof back to Alice

In [13]:
response = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof)
print(response)

{'state': 'presentation_sent', 'connection_id': '35370789-764f-443a-9526-94044dbdc702', 'role': 'prover', 'updated_at': '2020-09-03 12:01:55.957975Z', 'initiator': 'external', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_name_uuid': {'name': 'name', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_skill_uuid': {'name': 'skill', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_country_uuid': {'name': 'country'}}, 'requested_predicates': {'0_age_GE_uuid': {'name': 'age', 'p_type': '>=', 'p_value': 21, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '1012453674509974792060177'}, 'thread_id': 'd0dec549-b119-4f05-9384-1edfcd634bc5', 'created_at': '2020-09-03 11:59:11.295947Z', 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'name': '93006290325627508022776103386395994712401809437930957652111221015872244345185', 'skill': '24214165878

Handle present proof
{'state': 'presentation_sent', 'connection_id': '35370789-764f-443a-9526-94044dbdc702', 'role': 'prover', 'updated_at': '2020-09-03 12:01:55.957975Z', 'initiator': 'external', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_name_uuid': {'name': 'name', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_skill_uuid': {'name': 'skill', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_country_uuid': {'name': 'country'}}, 'requested_predicates': {'0_age_GE_uuid': {'name': 'age', 'p_type': '>=', 'p_value': 21, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '1012453674509974792060177'}, 'thread_id': 'd0dec549-b119-4f05-9384-1edfcd634bc5', 'created_at': '2020-09-03 11:59:11.295947Z', 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'name': '93006290325627508022776103386395994712401809437930957652111221015872244345185', 

Handle present proof
{'state': 'presentation_acked', 'connection_id': '35370789-764f-443a-9526-94044dbdc702', 'role': 'prover', 'updated_at': '2020-09-03 12:02:18.252474Z', 'initiator': 'external', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_name_uuid': {'name': 'name', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_skill_uuid': {'name': 'skill', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_country_uuid': {'name': 'country'}}, 'requested_predicates': {'0_age_GE_uuid': {'name': 'age', 'p_type': '>=', 'p_value': 21, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '1012453674509974792060177'}, 'thread_id': 'd0dec549-b119-4f05-9384-1edfcd634bc5', 'created_at': '2020-09-03 11:59:11.295947Z', 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'name': '93006290325627508022776103386395994712401809437930957652111221015872244345185',

## 11. Continue in the verifier notebook

The verifier needs to check the proof send by Bob is verifiable.

## End of Tutorial

Be sure to terminate the controller.

In [ ]:
response = await agent_controller.terminate()